In [105]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
import sklearn.externals 
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [55]:
df1=pd.read_csv(r"C:\Mba notes\novels\StudentsPerformance.csv")
df1

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75
...,...,...,...,...,...,...,...,...
995,female,group E,master's degree,standard,completed,88,99,95
996,male,group C,high school,free/reduced,none,62,55,55
997,female,group C,high school,free/reduced,completed,59,71,65
998,female,group D,some college,standard,completed,68,78,77


In [56]:
s1=list(df1["lunch"].sample(50))
s2=list(df1['math score'].sample(50))

In [57]:
def random_nulls(column,sample):
    for i in range(len(sample)):
        if column[i] in sample:
            column[i]=None
random_nulls(df1['lunch'],s1)
random_nulls(df1['math score'],s2)


<ipython-input-57-ff654ddae2f3>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  column[i]=None


In [58]:
df1.isnull().sum()

gender                          0
race/ethnicity                  0
parental level of education     0
lunch                          50
test preparation course         0
math score                     30
reading score                   0
writing score                   0
dtype: int64

In [94]:
categorical_names=[i for i in df1.columns if df1[i].dtype=="O"]
numerical_names=[i for i in df1.columns if df1[i].dtype=="float64" or df1[i].dtype=='int64']
categorical_cols=df1[categorical_names]
numerical_cols=df1[numerical_names]

numerical_transformer=SimpleImputer(strategy='constant')
categorical_transformer=Pipeline(steps=
                                [('imputer',SimpleImputer(strategy='constant')),
                                ('one hot encoding',OneHotEncoder(handle_unknown='ignore'))])
preprocessor=ColumnTransformer(transformers=[
    ('num',numerical_transformer,numerical_cols),
    ('cat',categorical_transformer,categorical_cols)
])

In [95]:
numerical_cols.isnull().sum()

math score       30
reading score     0
writing score     0
dtype: int64

In [99]:
df1.isnull().sum()

gender                          0
race/ethnicity                  0
parental level of education     0
lunch                          50
test preparation course         0
math score                     30
reading score                   0
writing score                   0
dtype: int64

In [103]:
df1['total score']=df1['math score']+df1['reading score']+df1['writing score']

In [112]:
x_train,_x_test,y_train,y_test=train_test_split(df1.drop(['total score'],1),df1['total score'],test_size=0.8,random_state=0)

<ipython-input-112-c20a8fbc50f5>:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  x_train,_x_test,y_train,y_test=train_test_split(df1.drop(['total score'],1),df1['total score'],test_size=0.8,random_state=0)


In [115]:
y_train

152    195.0
528    117.0
696    247.0
621    181.0
461    153.0
       ...  
835    198.0
192    192.0
629    150.0
559    201.0
684    196.0
Name: total score, Length: 200, dtype: float64

In [117]:
model=RandomForestClassifier(n_estimators=100,random_state=0)
my_pipeline=Pipeline(steps=[('preprocessor',preprocessor),
                           ('model',model)])
my_pipeline.fit(df1.drop('writing score',1),df1['writing score'])

<ipython-input-117-43cd0735fac8>:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  my_pipeline.fit(df1.drop('writing score',1),df1['writing score'])


ValueError: No valid specification of the columns. Only a scalar, list or slice of all integers or all strings, or boolean mask is allowed

In [21]:
df2=pd.read_csv("C:/Mba notes/novels/AWS/Harvest/Crop_recommendation.csv")

In [23]:
x_train,x_test,y_train,y_test=train_test_split(df2.drop('label',1),df2['label'],random_state=3)

<ipython-input-23-447d4acfaab7>:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  x_train,x_test,y_train,y_test=train_test_split(df2.drop('label',1),df2['label'],random_state=3)


In [24]:
x_train.isnull().sum()

N              0
P              0
K              0
temperature    0
humidity       0
ph             0
rainfall       0
dtype: int64

In [25]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import make_column_transformer,ColumnTransformer

OHE=OneHotEncoder()
SI=SimpleImputer()

ct=make_column_transformer(
(OHE,["gender",'parental level of education']),
(SI,['math score']))
ct.fit_transform(df1).shape

c_cols=df1[['lunch','parental level of education']]
n_cols=df1['math score']

CT=ColumnTransformer(
         transformers=[
             ("categorical",OHE,c_cols),
             ('numerical',SI,n_cols)
         ])

In [47]:
pipe1=Pipeline([("scaler1",StandardScaler()),
              ("pca",PCA(n_components=2)),
              ('classifier',LogisticRegression(random_state=1,max_iter=3000))])
pipe2=Pipeline([("scaler1",StandardScaler()),
              ("pca",PCA(n_components=2)),
              ('classifier',DecisionTreeClassifier())])
pipe3=Pipeline([("scaler1",StandardScaler()),
              ("pca",PCA(n_components=2)),
              ('classifier',RandomForestClassifier())])

In [48]:
pipelines=[pipe1,pipe2,pipe3]
for i in pipelines:
    i.fit(x_train,y_train)
    print(i.score(x_test,y_test))

0.48
0.41818181818181815
0.48


In [33]:
pipe1.score(x_test,y_test)

0.48